In [ ]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import agentops
from pydantic import BaseModel, Field
from typing import List
from tavily import TavilyClient
from scrapegraph_py import Client
from dotenv import load_dotenv
import os
import json

## Setting up an API key

In [ ]:
load_dotenv()
agentops_key = os.getenv("AGENTOPS_API_KEY")
print("Loaded AgentOps key:", agentops_key[:10] + "...") 


agentops.init(
    api_key=agentops_key,
    skip_auto_end_session=True,
    default_tags=['crewai']
)


🔑 Loaded AgentOps key: 07e05ec1-c...


🖇 AgentOps: You're on the agentops free plan 🤔
--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\logging\__init__.py", line 1113, in emit
    stream.write(msg + self.terminator)
  File "C:\Program Files\Python311\Lib\encodings\cp1256.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f914' in position 66: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\manage AI agents\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\user\OneDrive - University of Prince Mugrin\سطح المكتب\manage AI agents\venv\Lib\site-packages\traitlets\config\applicat

## Setting up LLM Models



In [ ]:
output_dir = r"./output"
os.makedirs(output_dir, exist_ok=True)


groq_llm = LLM(
    model="groq/llama-3.1-8b-instant",
    temperature=0.1
)
groq_llm2 = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.1
)


search_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
scrape_client = Client(api_key=os.getenv("SGAI_API_KEY"))




## Search_Queries_Recommendation


In [ ]:
no_keywords = 10
class SuggestedSearchQueries(BaseModel):

     queries: List[str] = Field(...,title="Suggested search queries to be passed to the search engine",
                               min_length=1, max_length=no_keywords)




search_queries_recommendation_agent=Agent(
        role="search_queries_recommendation_agent",
        goal="/n".join([
              "To provide a list of suggested search queries to be passed to the search engine.",
              "Generating multiple variations of search queries — including keyword-based looking for specific items"
         ]),
        backstory="The agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context provided.",
        llm=groq_llm,
        verbose=True
)

search_queries_recommendation_Task=Task(
    description="\n".join([
        "Ohay is looking to buy {product_name} at the best prices (value for a price strategy)",
        "The campany target any of these websites to buy from: {websites_list}",
        "The stores must sell the product in {country_name}",
        "Generate at maximum {no_keywords} queries.",
        "Search keywords must contains specific brands, types or technologies. Avoid general keywords.",
        "The search query must reach an ecommerce webpage for product, and not a blog or listing page."]),
    expected_output="A JSON object containing a list of suggested search queries.",
    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir, "step_1_suggested_search_queries.json"),
    agent=search_queries_recommendation_agent
    )





## Agent Search_Engine

In [5]:

class SignleSearchResult(BaseModel):
    title: str
    url: str = Field(..., title="the page url")
    content: str
    score: float
    rating : float
    search_query: str
    
class AllSearchResult(BaseModel):
    results: List[SignleSearchResult]


@tool
def search_engine_tool(query:str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""

    return search_client.search(query)



search_engine_agent=Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is designed to help in looking for products by searching for products based on the suggested search queries.",
    llm=groq_llm2,
    verbose=True,
    tools=[search_engine_tool]
)


search_engine_task=Task(
    description="\n".join([
        "The task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries.",
        "Ignore any susbicious links or not an ecommerce single product website link.",
        "Ignore any search results with confidence score less than ({score_th}) and a customer rating (score) lower than ({score_ra})",
        "The search results will be used to compare prices of products from different websites.",

    ]),
    expected_output="A JSON object containing the search results.",
    output_json=SignleSearchResult,
    output_file=os.path.join(output_dir, "step_2_search_engine.json"),
    context=[search_queries_recommendation_Task],
    agent=search_engine_agent
)

##  Web_Scraping_Agent

In [6]:
class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str


class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the product page")
    product_title: str = Field(..., title="The title of the product")
    product_image_url: str = Field(..., title="The url of the product image")
    product_url: str = Field(..., title="The url of the product")
    product_current_price: float = Field(..., title="The current price of the product")
    product_original_price: float = Field(title="The original price of the product before discount. Set to None if no discount", default=None)
    product_discount_percentage: float = Field(title="The discount percentage of the product. Set to None if no discount", default=None)
    product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)
    agent_recommendation_rank: int = Field(..., title="The rank of the product to be considered in the final procurement report. (out of 5, Higher is Better) in the recommendation list ordering from the best to the worst")
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this product to the company, compared to other products.")


class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]


@tool
def web_scraping_tool(page_url: str):
    """
        An AI Tool to help an agent to scrape a web page

        Example:
        web_scraping_tool(
            page_url="https://www.noon.com/saudi-ar/4-in-1-automatic-espresso-machine-1800w-2-6l-cappuccino-latte-espresso-grind-coffee-machine-with-automatic-milk-frother-20-bar-pump-pressure-touchscreen-coffee-maker-for-home-and-office-sk-04032/ZE622C99C49629E605245Z/p/?o=ze622c99c49629e605245z-1&shareId=a8f2656d-996c-42b1-a4f1-0ec14d7ae19c"
        )
        """

    schema_json = json.dumps(SingleExtractedProduct.model_json_schema(), indent=2)

    details = scrape_client.smartscraper(
        website_url=page_url,
        user_prompt=f"Extract ```json\n"+ {schema_json}+ "```\n From the web page"
    )

    return {
         "product": [details]
    }


scraping_agent = Agent(
    role="Web scraping agent",
    goal="To extract details from any website",
    backstory="The agent is designed to help in looking for required values from any website url. These details will be used to decide which best product to buy.",
    llm=groq_llm2,
    tools=[web_scraping_tool],
    verbose=True,
)

scraping_task = Task(
    description="\n".join([
        "The task is to extract product details from any ecommerce store page url.",
        "The task has to collect results from multiple pages urls.",
        "Collect the best products from the search results.",
    ]),
    expected_output="A JSON object containing products details",
    output_json=AllExtractedProducts,
    output_file=os.path.join(output_dir, "step_3_search_results.json"),
    context=[search_engine_task],
    agent=scraping_agent
)

C:\Users\user\AppData\Local\Temp\ipykernel_23628\2439442555.py:14: PydanticDeprecatedSince20: `min_items` is deprecated and will be removed, use `min_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)
C:\Users\user\AppData\Local\Temp\ipykernel_23628\2439442555.py:14: PydanticDeprecatedSince20: `max_items` is deprecated and will be removed, use `max_length` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important specs to compare.", min_items=1, max_items=5)


## Procurement_Report_Author_Agent

In [7]:
about_company = "ohay is a company that provides AI solutions to help websites refine their search and recommendation systems."

company_context = StringKnowledgeSource(
    content=about_company
)

In [8]:
procurement_report_author_agent = Agent(
    role="Procurement Report Author Agent",
    goal="To generate a professional HTML page for the procurement report",
    backstory="The agent is designed to assist in generating a professional HTML page for the procurement report after looking into a list of products.",
    llm=groq_llm2,
    verbose=True,
   
)

procurement_report_author_task =Task(
    description="\n".join([
        "The task is to generate a professional HTML page for the procurement report.",
        "You have to use Bootstrap CSS framework for a better UI.",
        "Use the provided context about the OHAY company to make a specialized report.",
        "The report will include the search results and prices of products from different websites.",
        "The report should be structured with the following sections:",
        "1. Executive Summary: A brief overview of the procurement process and key findings.",
        "2. Introduction: An introduction to the purpose and scope of the report.",
        "3. Methodology: A description of the methods used to gather and compare prices.",
        "4. Findings: Detailed comparison of prices from different websites, including tables and charts.",
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations.",
        "6. Recommendations: Suggestions for procurement based on the analysis.",
        "7. Conclusion: A summary of the report and final thoughts.",
        "8. Appendices: Any additional information, such as raw data or supplementary materials."
    ]),
    expected_output="A professional HTML page for the procurement report.",
    output_file=os.path.join(output_dir, "step_4_procurement_report.html"),
    context=[scraping_task],
    agent=procurement_report_author_agent
)

## Procurement_Report_Critic_Agent

In [15]:
procurement_report_critic_agent = Agent(
    role="Procurement Quality Assurance Expert",
    goal="Critically review procurement reports for accuracy, completeness, and quality",
    backstory="""You are a meticulous procurement analyst with 15 years of experience. 
    Your expertise includes:
    - Identifying gaps in market analysis
    - Spotting missing cost factors
    - Checking supplier evaluation completeness
    - Ensuring risk assessment thoroughness
    - Validating data consistency and accuracy
    
    You are known for being thorough, critical, and constructive. You never accept 
    reports at face value and always look for ways to improve them.""",
    llm=groq_llm,
    verbose=True,
    reasoning=True,  # Enable reasoning
    max_reasoning_attempts=3
)



# NEW: Critique Task
critique_task = Task(
    description="\n".join([
        """Thoroughly review and critique the draft procurement report.
        
        Provide specific, constructive feedback on:
        
        **Accuracy & Completeness:**
        - Are all cost factors included?
        - Is the market analysis comprehensive?
        - Are there any factual inaccuracies?
        - Is the data interpretation correct?
        
        **Structure & Clarity:**
        - Is the report well-organized?
        - Are recommendations clear and actionable?
        - Is the executive summary effective?
        
        **Gaps & Improvements:**
        - What important information is missing?
        - What sections need more depth?
        - Are there better alternatives not considered?
        
        Provide detailed, specific feedback that the author can use to improve the report."""
    ]),
    agent=procurement_report_critic_agent,
    context=[procurement_report_author_task],
    expected_output="Detailed critique with specific improvement suggestions"
)

# NEW: Revision Task
revision_task = Task(
    description="\n".join([
        """
        Revise and improve the procurement report based on the detailed critique provided.
        
        Address all points raised in the critique:
        - Fix any factual inaccuracies
        - Fill identified information gaps
        - Improve structure and clarity
        - Strengthen recommendations
        - Enhance executive summary
        
        Ensure the final report is polished, comprehensive, and meets the highest quality standards.
        Incorporate the feedback while maintaining your professional writing style.
    """]),
    agent=procurement_report_author_agent,
    context=[procurement_report_author_task, critique_task],
    expected_output="Final, polished procurement report",
    output_file=os.path.join(output_dir, "step_4_updet_procurement_report.html")
    
)

In [16]:
Ohay_crew=Crew(
    agents=[search_queries_recommendation_agent,search_engine_agent,scraping_agent,procurement_report_author_agent,procurement_report_critic_agent],
    tasks=[search_queries_recommendation_Task,search_engine_task,scraping_task,procurement_report_author_task,critique_task,revision_task],
    process=Process.sequential,
    knowledge_sources=[company_context],
    memory=True
)

relsut=Ohay_crew.kickoff(
        inputs={
            "product_name": "coffee machine for the office",
            "websites_list":  ["www.amazon.eg", "www.jumia.com.eg", "www.noon.com/saudi-en"],
            "country_name": "Saudi",
            "no_keywords": 10,
            "score_th":0.10,
            "score_ra":3.5

        }
)


 
[2025-11-02 11:11:25][ERROR]: Failed to upsert documents: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: search_queries_recommendation_agent
## Task: Ohay is looking to buy coffee machine for the office at the best prices (value for a price strategy)
The campany target any of these websites to buy from: ['www.amazon.eg', 'www.jumia.com.eg', 'www.noon.com/saudi-en']
The stores must sell the product in Saudi
Generate at maximum 10 queries.
Search keywords must contains specific brands, types or technologies. Avoid general keywords.
The search query must reach an ecommerce webpage for product, and not a blog or listing page.


# Agent: search_queries_recommendation_agent
## Final Answer: 
Thought: I now can give a great answer

{
  "queries": [
    "Breville coffee machine Saudi Arabia",
    "De'Longhi espresso machine Amazon Egypt",
    "Nespresso coffee machine Jumia Egypt",
    "Miele coffee machine Noon Saudi Arabia",
    "Jura coffee machine best price Saudi Arabia",
    "Saeco coffee machine with milk frother Amazon Egypt",
    "Tassimo coffee machine with coffee beans Jumia E

ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



...

ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during short_term search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities search: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'


# Agent: Procurement Quality Assurance Expert
## Task: Thoroughly review and critique the draft procurement report.

        Provide specific, constructive feedback on:

        **Accuracy & Completeness:**
        - Are all cost factors included?
        - Is the market analysis comprehensive?
        - Are there any factual inaccuracies?
        - Is the data interpretation correct?

        **Structure & Clarity:**
        - Is the report well-organized?
        - Are recommendations clear and actionable?
        - Is the executive summary effective?

        **Gaps & Improvements:**
        - What important information is missing?
        - What sections need more depth?
        - Are there better alternatives not considered?

        Provide detailed, specific feedback that the author can use to improve the report.


# Agent: Procurement Quality Assurance Expert
## Final Answer: 
**

**Accuracy & Completeness:**

1. **Missing cost factors:** The report only considers the current p

ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyw

# Agent: Procurement Report Author Agent
## Task: 
        Revise and improve the procurement report based on the detailed critique provided.

        Address all points raised in the critique:
        - Fix any factual inaccuracies
        - Fill identified information gaps
        - Improve structure and clarity
        - Strengthen recommendations
        - Enhance executive summary

        Ensure the final report is polished, comprehensive, and meets the highest quality standards.
        Incorporate the feedback while maintaining your professional writing style.
    


# Agent: Procurement Report Author Agent
## Final Answer: 
```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Procurement Report for OHAY Company</title>
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css">
    <script src="https://code.jquery.com/jquery-3

ERROR:root:Error during short_term save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
ERROR:root:Error during entities save: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'
